# 使用 AgentExecutor (旧版) 构建 Agent

:::important
本节将介绍如何使用旧版的 LangChain AgentExecutor 进行构建。虽说它们适合入门，但一旦超越某个阶段，你可能会发现它们在灵活性和控制力方面无法满足你的需求。对于更高级的 Agent，我们建议你查阅 [LangGraph Agents](/docs/concepts/architecture/#langgraph) 或 [迁移指南](/docs/how_to/migrate_agent/)。
:::

语言模型本身无法执行操作，它们只会输出文本。
LangChain 的一个重要用例是创建 **[agents](/docs/concepts/agents/)**。
Agents 是利用 LLM 作为推理引擎，来决定执行何种操作以及这些操作的输入是什么的系统。
这些操作的结果随后可以反馈给 Agent，它会决定是否需要执行更多操作，或者是否可以结束。

在本教程中，我们将构建一个能够与多个不同工具交互的 Agent：其中一个是本地数据库，另一个是搜索引擎。你将能够向这个 Agent 提问，观察它调用工具，并与它进行对话。

## 概念

我们将涵盖的概念包括：
- 使用 [语言模型](/docs/concepts/chat_models)，特别是它们的工具调用能力
- 创建一个 [Retriever](/docs/concepts/retrievers) 以向我们的 Agent 暴露特定信息
- 使用搜索 [Tool](/docs/concepts/tools) 在网上查找内容
- [`Chat History`](/docs/concepts/chat_history)，它允许聊天机器人“记住”过去的交互，并在回应后续问题时将其考虑在内。
- 使用 [LangSmith](https://docs.smith.langchain.com/) 进行应用程序的调试和追踪

## 设置

### Jupyter Notebook

本指南（以及文档中的大多数其他指南）使用 [Jupyter notebooks](https://jupyter.org/)，并假设读者也在使用。Jupyter notebooks 非常适合学习如何使用 LLM 系统，因为事情常常会出错（意外输出、API 宕机等），而在交互式环境中学习指南是更好地理解它们的绝佳方式。

本教程及其他教程最好在 Jupyter notebook 中运行。请参阅 [此处](https://jupyter.org/install) 查看安装说明。

### 安装

要安装 LangChain，请运行：

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install langchain</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>

有关更多详细信息，请参阅我们的 [安装指南](/docs/how_to/installation)。

### LangSmith

你用 LangChain 构建的许多应用程序将包含多个步骤以及多次 LLM 调用。
随着这些应用程序变得越来越复杂，能够检查你的 Chain 或 Agent 内部到底发生了什么变得至关重要。
做到这一点最好的方法就是使用 [LangSmith](https://smith.langchain.com)。

在上面的链接注册后，请确保设置你的环境变量以开始记录追踪：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

或者，如果你在 notebook 中，你可以用以下方式设置它们：

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

## 定义工具

我们首先需要创建想要使用的工具。我们将使用两个工具：[Tavily](/docs/integrations/tools/tavily_search)（用于在线搜索）以及一个我们稍后创建的本地索引的检索器。

### [Tavily](/docs/integrations/tools/tavily_search)

LangChain 内置了一个工具，可以轻松地将 Tavily 搜索引擎用作工具。
请注意，这需要一个 API 密钥——它们提供免费套餐，但如果你没有或不想创建一个，可以随时忽略此步骤。

创建 API 密钥后，你需要将其导出为：

```bash
export TAVILY_API_KEY="..."
```

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [6]:
search = TavilySearchResults(max_results=2)

In [7]:
search.invoke("what is the weather in SF")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1714000492, 'localtime': '2024-04-24 16:14'}, 'current': {'last_updated_epoch': 1713999600, 'last_updated': '2024-04-24 16:00', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 330, 'wind_dir': 'NNW', 'pressure_mb': 1018.0, 'pressure_in': 30.06, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 72, 'cloud': 100, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 5.0, 'gust_mph': 14.8, 'gust_kph': 23.8}}"},
 {'url': 'https://www.weathertab.com/en/c/e/04/united-states/california/san-francisco/',
  'content': 'San Francisco Weather Forecast for Apr 2024 - Risk of Rain Graph. R

### Retriever

我们还将创建一个检索器来处理我们自己的数据。有关此处每个步骤的更深入的解释，请参阅[此教程](/docs/tutorials/rag)。

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [9]:
retriever.invoke("how to upload a dataset")[0]

Document(page_content='# The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, Example } from \'langsmith\';import { runOnDataset } from \'langchain/smith\';import { EvaluationResult } from \'langsmith/evaluation\';const client = new Client();// Define dataset: these are your test casesconst datasetName = "Sample Dataset";const dataset = await client.createDataset(datasetName, {    description: "A sample dataset in LangSmith."});await client.createExamples({    inputs: [        { postfix: "to LangSmith" },        { postfix: "to Evaluations in LangSmith" },    ],    outputs: [        { output: "Welcome to LangSmith" },        { output: "Welcome to Evaluations in LangSmith" },    ],    datasetId: dataset.id,});// Define your evaluatorconst exactMatch = async ({ run, example }: {

现在我们已经填充了我们将要进行检索的索引，我们可以轻松地将其变成一个工具（代理程序需要正确使用它的格式）。

In [10]:
from langchain.tools.retriever import create_retriever_tool

In [11]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

### 工具

现在我们已经创建了两者，我们可以创建一个下游将要使用的工具列表。

In [12]:
tools = [search, retriever_tool]

## 使用语言模型

接下来，我们将学习如何使用语言模型调用工具。LangChain 支持许多不同的语言模型，您可以互换使用它们——在下方选择您想使用的模型！

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs overrideParams={{openai: {model: "gpt-4"}}} />

In [4]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

您可以向语言模型传递消息列表来调用它。默认情况下，响应是 `content` 字符串。

In [13]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

现在我们可以了解启用此模型进行工具调用的情况了。为了实现这一点，我们使用 `.bind_tools` 来使语言模型了解这些工具。

In [14]:
model_with_tools = model.bind_tools(tools)

现在我们可以调用模型了。让我们先用一个普通消息调用它，看看它如何回应。我们可以同时查看 `content` 字段和 `tool_calls` 字段。

In [18]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


现在，让我们尝试使用一些期望调用工具的输入来调用它。

In [19]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_4HteVahXkRAkWjp6dGXryKZX'}]


我们可以看到，现在没有内容了，但有一个工具调用！它要求我们调用 Tavily Search 工具。

这还不是在调用该工具——它只是在告诉我们要调用它。为了实际调用它，我们需要创建我们的代理。

## 创建代理

现在我们已经定义了工具和 LLM，就可以创建代理了。我们将使用一个工具调用代理——关于这种类型的代理以及其他选项的更多信息，请参阅[本指南](/docs/concepts/agents/)。

我们可以首先选择用于指导代理的提示。

如果你想查看此提示的内容并访问 LangSmith，可以转到：

[https://smith.langchain.com/hub/hwchase17/openai-functions-agent](https://smith.langchain.com/hub/hwchase17/openai-functions-agent)

In [20]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

现在，我们可以初始化代理（agent）的 LLM、提示（prompt）和工具（tools）。代理负责接收输入并决定采取什么行动。至关重要的是，代理本身并不执行这些行动——这些行动将由 AgentExecutor 来执行（下一步骤）。有关如何理解这些组件的更多信息，请参阅我们的[概念指南](/docs/concepts/agents)。

请注意，我们传递的是 `model`，而不是 `model_with_tools`。这是因为 `create_tool_calling_agent` 会在后台为我们调用 `.bind_tools`。

In [23]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)

最后，我们将 Agent（大脑）和工具组合在 AgentExecutor 中（它将反复调用 Agent 并执行工具）。

In [24]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

## 运行代理

现在我们可以对一些查询运行代理了！请注意，目前这些都是**无状态**查询（它不会记住之前的交互）。

首先，让我们看看在不需要调用工具时它的响应：

In [25]:
agent_executor.invoke({"input": "hi!"})

{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

为了准确了解幕后发生的情况（并确保它没有调用工具），我们可以查看一下 LangSmith 跟踪： [LangSmith trace](https://smith.langchain.com/public/8441812b-94ce-4832-93ec-e1114214553a/r)

现在让我们在它应该调用检索器的示例上进行尝试。

In [26]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith is a platform that aids in building production-grade Language Learning Model (LLM) applications. It can assist with testing in several ways:\n\n1. **Monitoring and Evaluation**: LangSmith allows close monitoring and evaluation of your application. This helps you to ensure the quality of your application and deploy it with confidence.\n\n2. **Tracing**: LangSmith has tracing capabilities that can be beneficial for debugging and understanding the behavior of your application.\n\n3. **Evaluation Capabilities**: LangSmith has built-in tools for evaluating the performance of your LLM. \n\n4. **Prompt Hub**: This is a prompt management tool built into LangSmith that can help in testing different prompts and their responses.\n\nPlease note that to use LangSmith, you would need to install it and create an API key. The platform offers Python and Typescript SDKs for utilization. It works independently and does not require th

让我们看一下 [LangSmith trace](https://smith.langchain.com/public/762153f6-14d4-4c98-8659-82650f860c62/r)，以确保它确实调用了该工具。

现在我们来试试需要调用搜索工具的一个：

In [27]:
agent_executor.invoke({"input": "whats the weather in sf?"})

{'input': 'whats the weather in sf?',
 'output': 'The current weather in San Francisco is partly cloudy with a temperature of 16.1°C (61.0°F). The wind is coming from the WNW at a speed of 10.5 mph. The humidity is at 67%. [source](https://www.weatherapi.com/)'}

我们可以查看 [LangSmith trace](https://smith.langchain.com/public/36df5b1a-9a0b-4185-bae2-964e1d53c665/r) 来确保它有效地调用了搜索工具。

## 添加记忆

如前所述，该代理是无状态的。这意味着它不记得之前的交互。为了给它记忆，我们需要传入先前的 `chat_history`。注意：由于我们使用的提示，它必须被命名为 `chat_history`。如果我们使用不同的提示，可以更改变量名。

In [28]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})

{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [29]:
from langchain_core.messages import AIMessage, HumanMessage

In [30]:
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)

{'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'input': "what's my name?",
 'output': 'Your name is Bob. How can I assist you further?'}

如果我们想自动跟踪这些消息，我们可以将其包装在 `RunnableWithMessageHistory` 中。有关如何使用它的更多信息，请参阅[本指南](/docs/how_to/message_history)。

In [36]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

因为我们有多个输入，所以需要指定两件事：

- `input_messages_key`: 用于添加到对话历史记录的输入键。
- `history_messages_key`: 用于将加载的消息添加到的键。

In [37]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [38]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    config={"configurable": {"session_id": "<foo>"}},
)

{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [39]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    config={"configurable": {"session_id": "<foo>"}},
)

{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob.'}

示例 LangSmith trace：https://smith.langchain.com/public/98c8d162-60ae-4493-aa9f-992d87bd0429/r

## 结论

本次教程到此结束！在这个快速入门指南中，我们学习了如何创建一个简单的代理。代理是一个复杂的话题，还有很多内容值得学习！

:::important
本节介绍了如何使用 LangChain Agents 进行构建。它们对于入门来说很不错，但达到一定程度后，您很可能会需要它们不提供的灵活性和控制力。要开发更高级的代理，我们建议您查看 [LangGraph](/docs/concepts/architecture/#langgraph)
:::

如果您想继续使用 LangChain 代理，以下是一些优秀的进阶指南：

- [如何使用 LangGraph 的内置版本 `AgentExecutor`](/docs/how_to/migrate_agent)
- [如何创建自定义代理](https://python.langchain.com/v0.1/docs/modules/agents/how_to/custom_agent/)
- [如何从代理流式传输响应](https://python.langchain.com/v0.1/docs/modules/agents/how_to/streaming/)
- [如何从代理返回结构化输出](https://python.langchain.com/v0.1/docs/modules/agents/how_to/agent_structured/)